In [20]:
import pandas as pd
import numpy as np
import os
import glob
import xml.etree.ElementTree as ET 
import splitfolders

In [21]:
path_an = "annotations/"

In [22]:
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "name":[],    
            "file":[],
            "width":[],
            "height":[],
           }

for anno in glob.glob(path_an+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]                        
                

In [23]:
data=pd.DataFrame(dataset)
data

,xmin,ymin,xmax,ymax,name,file,width,height
0,62,194,160,320,without_mask,maksssksksss299,301,400
1,43,169,149,308,without_mask,maksssksksss528,301,400
2,48,107,218,304,mask_weared_incorrect,maksssksksss272,275,400
3,28,78,43,99,with_mask,maksssksksss514,400,267
4,160,66,176,83,with_mask,maksssksksss514,400,267
...,...,...,...,...,...,...,...,...
4067,271,73,278,82,without_mask,maksssksksss294,400,241
4068,236,91,243,99,without_mask,maksssksksss294,400,241
4069,236,76,243,83,without_mask,maksssksksss294,400,241
4070,264,76,268,82,with_mask,maksssksksss294,400,241


In [ ]:
data['class']=data['name'].map({'with_mask':0, 'mask_weared_incorrect':1, 'without_mask':2})
data['Xcent']=(data['xmin']+data['xmax'])/(2*data['width'])
data['Ycent']=(data['ymin']+data['ymax'])/(2*data['height'])
data['boxW']=(data['xmax']-data['xmin'])/data['width']
data['boxH']=(data['ymax']-data['ymin'])/data['height']

In [ ]:
for index, row in data.iterrows():
    print(index)
    im = Image.open("images/{}.png".format(data.loc[index,'file']))
    im = im.crop((data.loc[index,'xmin'], data.loc[index,'ymin'], data.loc[index,'xmax'], data.loc[index,'ymax'])) 
    if data.loc[index,'class'] == 0:
        im.save("Processed Images/with mask/{}.png".format(index)) # saves the image
    elif data.loc[index,'class'] == 1:
        im.save("Processed Images/mask weared incorrect/{}.png".format(index))
    elif data.loc[index,'class'] == 2:
        im.save("Processed Images/without mask/{}.png".format(index))

In [ ]:
input_folder = "Processed Data"
output = "Processed Images" 
splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.